# Data Preprocessing

In [58]:
# Input: text corpus
# step 1: extract key phrases (autophrase)
# step 2: generate embeddings

## Extract Key Phrases

In [59]:
# details here: https://github.com/rit-git/meg-kb/tree/main/src/keyword_extraction

In [60]:
#change to keyword extractor directory
%cd ../keyword_extraction/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/keyword_extraction


In [61]:
!chmod +x ./corpusProcess.sh

In [62]:
# select the dataset and thread no
data = 'sample-meg-ac'
thread = 8

In [63]:
# process corpus and generate key prhases
!./corpusProcess.sh $data $thread

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/keyword_extraction
===Corpus Name: sample-meg-ac===
===Current Path: /mnt/efs/shared/meg_shared_scripts/meg-kb/src/keyword_extraction===
===Cleaning input corpus===
===Running AutoPhrase===
make: Nothing to be done for 'all'.
===RAW_TRAIN: ../../../data/sample-meg-ac/source/corpus.clean.txt===
auto_phrase.sh parameters: sample-meg-ac ../../../data/sample-meg-ac/source/corpus.clean.txt 10 data/EN/wiki_quality.txt 8
===Compilation===
===Tokenization===
Current step: Tokenizing input file...
real	0m0.627s
user	0m1.300s
sys	0m0.128s
Detected Language: EN
Current step: Tokenizing wikipedia phrases...
No provided expert labels.
===Part-Of-Speech Tagging===
===AutoPhrasing===
=== Current Settings ===
Iterations = 2
Minimum Support Threshold = 10
Maximum Length Threshold = 6
POS-Tagging Mode Disabled
Discard Ratio = 0.050000
Number of threads = 8
Labeling Method = DPDN
	Auto labels from knowledge bases
	Max Positive Samples = -1
Loading data...
# o

In [64]:
# copy these results to sample-meg-pt
!cp -r ../../data/sample-meg-ac ../../data/sample-meg-pt

# Generate Embeddings

In [65]:
# details here: https://github.com/rit-git/meg-kb/tree/main/src/concept_learning

In [66]:
#change to concept learning directory
%cd ../concept_learning/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning


## Sentence Embedding

In [67]:
!python compute_keyphrase_embeddings.py -m bert-base-uncased -et ac -d ../../data/$data/intermediate -c 750

computing entity-wise embedding: 100%|████████| 175/175 [00:04<00:00, 41.24it/s]
Saving embedding


## Concatenated Token Embedding

In [68]:
data_pt = 'sample-meg-pt'

In [69]:
!python compute_keyphrase_embeddings.py -m bert-base-uncased -et pt -d ../../data/$data_pt/intermediate -c 750

computing entity-wise embedding: 100%|████████| 175/175 [00:03<00:00, 53.71it/s]
Saving embedding


## Token Embedding

In [70]:
# change directory to autophrase
%cd ../tools/AutoPhrase

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/tools/AutoPhrase


In [71]:
data_corel = 'sample-corel'

In [72]:
!CUDA_VISIBLE_DEVICES=0 python extractBertEmbedding.py ../../../data/$data_corel/intermediate/ $thread

1
2021-05-07 20:21:04,464 : INFO : loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ubuntu/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2021-05-07 20:21:04,782 : INFO : loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/ubuntu/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2021-05-07 20:21:04,782 : INFO : Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embedding

# Expand Seed Entities (clustering)

In [73]:
# details here: https://github.com/rit-git/meg-kb/tree/main/src/concept_learning

In [74]:
#change to concept learning directory
%cd ../../concept_learning/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning


## knn sentence-embedding

In [75]:
clusters = 20
output = '../../data/'+data+'/intermediate/knn_'+str(clusters)+'.csv'
dim = 768

In [76]:
!python compute_concept_clusters.py -d ../../data/$data/intermediate/ -ca knn -s $clusters -dim $dim -o $output

finding nearest neighbors by entity: 100%|██| 175/175 [00:00<00:00, 7537.05it/s]


## knn token concatenated

In [77]:
clusters = 20
output = '../../data/'+data_pt+'/intermediate/knn_'+str(clusters)+'.csv'
dim = 3072

In [78]:
!python compute_concept_clusters.py -d ../../data/$data_pt/intermediate/ -ca knn -s $clusters -dim $dim -o $output

finding nearest neighbors by entity: 100%|██| 175/175 [00:00<00:00, 4095.89it/s]


## knn token

In [79]:
clusters = 20
output = '../../data/'+data_pt+'/intermediate/knn_'+str(clusters)+'.csv'
dim = 768

In [80]:
!python compute_concept_clusters.py -d ../../data/$data_corel/intermediate/ -ca knn -s $clusters -dim $dim -o $output

finding nearest neighbors by entity: 100%|██| 162/162 [00:00<00:00, 7408.57it/s]


# Mine Prompts

In [81]:
# Explore various techniques
# Get prompts "between" entities
# Get prompts by syntactic parsing
# Get prompts by paraphrasing
# Get prompts uisng AutoPrompt

# Retrieve Prompt Evidence

# Suggest Quality Prompts